In [11]:
import os
import requests
import pandas as pd
import datetime as dt
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

In [12]:
# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
def get_workday(date):
    date = pd.to_datetime(date)
    offset = max(1, (date.weekday() + 6) % 7 - 3)
    timedelta = dt.timedelta(offset)
    return pd.Timestamp((date - timedelta).date(),tz="America/New_York").isoformat()

In [14]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [19]:
# Format current date as ISO format
end_date = get_workday("today")
one_yrs = dt.timedelta(days=(365))
start_date = get_workday((pd.to_datetime("today") - one_yrs).date())
# Set the tickers
tickers = ["AAPL", "MSFT", "AMZN", "FB", "GOOGL", "BRK.B", "JNJ", "V", "PG", "BTC", "ETH", "XRP", "USDT", "BCH"]
# Set timeframe to '1D' for Alpaca API
timeframe = "1D"
# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
df_portfolio = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio.head()

AAPL                                      \
                             open     high      low   close    volume   
time                                                                    
2019-10-31 04:00:00+00:00  247.24  249.170  237.260  248.84  29023188   
2019-11-01 04:00:00+00:00  249.54  255.930  249.160  255.84  35281756   
2019-11-04 05:00:00+00:00  257.33  257.845  255.380  257.48  23618001   
2019-11-05 05:00:00+00:00  257.05  258.190  256.320  257.15  17738372   
2019-11-06 05:00:00+00:00  256.84  257.490  255.365  257.24  15328064   

                              AMZN                                           \
                              open      high        low      close   volume   
time                                                                          
2019-10-31 04:00:00+00:00  1775.99  1792.000  1771.4753  1777.1431  2328559   
2019-11-01 04:00:00+00:00  1788.01  1797.445  1785.2100  1791.4700  2447782   
2019-11-04 05:00:00+00:00  1801.50  1815.060  1801.0100  1805.2700  2513437   
2019-11-05 05:00:00+00:00  1809.16  1810.250  1794.0000  1801.7000  1650756   
2019-11-06 05:00:00+00:00  1801.00  1802.500  1788.5800  1795.9400  1790400   

                           ...        V                                   XRP  \
                           ...     open    high     low   close   volume open   
time                       ...                                                  
2019-10-31 04:00:00+00:00  ...  178.095  179.27  178.04  178.82  3409272  NaN   
2019-11-01 04:00:00+00:00  ...  180.130  181.32  179.56  180.90  4774444  NaN   
2019-11-04 05:00:00+00:00  ...  182.080  182.40  178.81  178.99  4696861  NaN   
2019-11-05 05:00:00+00:00  ...  179.070  179.56  175.18  176.35  6072064  NaN   
2019-11-06 05:00:00+00:00  ...  176.370  178.19  176.14  176.75  4597590  NaN   

                                                  
                          high  low close volume  
time                                              
2019-10-31 04:00:00+00:00  NaN  NaN   NaN    NaN  
2019-11-01 04:00:00+00:00  NaN  NaN   NaN    NaN  
2019-11-04 05:00:00+00:00  NaN  NaN   NaN    NaN  
2019-11-05 05:00:00+00:00  NaN  NaN   NaN    NaN  
2019-11-06 05:00:00+00:00  NaN  NaN   NaN    NaN  

[5 rows x 70 columns]